In [ ]:
import requests
import re
import concurrent.futures
import pandas as pd

URL = "https://pc.domains/basic/"  # API endpoint

def get_domain_price(domain):
    """Lấy giá tên miền từ API."""
    try:
        response = requests.get(URL + domain, verify=False, timeout=5)
        if response.status_code == 200:
            match = re.search(r'"EstimationModel",(\d+),', response.text)
            if match:
                return domain, int(match.group(1))
            else:
                return domain, None  # Không tìm thấy giá
        else:
            return domain, None  # Request thất bại
    except Exception:
        return domain, None  # Lỗi khi gọi API

def process_batch(batch_domains, max_workers=10):
    """Xử lý batch song song với max_workers luồng."""
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_domain = {executor.submit(get_domain_price, domain): domain for domain in batch_domains}
        for future in concurrent.futures.as_completed(future_to_domain):
            results.append(future.result())
    return results

# 📌 Đọc file batch
batch_file = "batches/batch_0.txt"  # Thay bằng file batch của bạn
with open(batch_file, "r") as f:
    domains = [line.strip() for line in f.readlines()]

# 📌 Gọi API song song
results = process_batch(domains, max_workers=20)

# 📌 Chuyển sang DataFrame & lưu kết quả
df = pd.DataFrame(results, columns=["domain", "price"])
df.to_csv("domain_prices.csv", index=False)
print(df.head())  # Kiểm tra kết quả


c:\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pc.domains'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pc.domains'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pc.domains'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS reque

In [ ]:
python get_price_tld.py uit.com